### Week 11
### DSC 650
### Abed Tabbalat

In [1]:
# Mounting drive
import os
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
os.chdir('/content/drive/My Drive/DSC650/Original/dsc650/assignments/assignment11')
!pwd

Mounted at /content/drive
/content/drive/My Drive/DSC650/Original/dsc650/assignments/assignment11


In [3]:
# Imports
import pandas as pd
import numpy as np
import keras
from keras import layers
import tensorflow as tf
import random
import sys


## Assignment 11

In [2]:
# Data imports

text = open('the_keeper_of_red_horse_pass.txt').read().lower()

print(f'Length: {len(text)}')

Length: 334537


In [4]:
# Set the random seed for reproducibility
np.random.seed(seed=15)

# Define the maximum length of each sequence and the step size
maxlen = 60
step = 3

# Initialize empty lists to store the sequences and their corresponding next characters
sentences = []
next_chars = []

# Iterate over the text, creating sequences of length `maxlen` with a step size of `step`
for i in range(0, len(text) - maxlen, step):
  # Append the current sequence to the `sentences` list
  sentences.append(text[i: i + maxlen])
  # Append the next character to the `next_chars` list
  next_chars.append(text[i + maxlen])

# Print the number of sequences and a sample sequence
print(f'Number of sequences: {len(sentences)}')
print(f'Sample: {sentences[10]}')

Number of sequences: 111493
Sample:  the keeper of red horse pass
this ebook is for the use of a


In [6]:
# Get the unique characters in the text and sort them
chars = sorted(list(set(text)))

# Create a dictionary that maps each character to its index in the `chars` list
char_indices = dict((char, chars.index(char)) for char in chars)

# Initialize arrays for the input and output
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

# Iterate over the sentences and characters in each sentence
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    # Encode each character in the input array using one-hot encoding
    x[i, t, char_indices[char]] = 1
    
  # Encode the next character in the output array using one-hot encoding
  y[i, char_indices[next_chars[i]]] = 1

In [7]:
# Define the model architecture
model = keras.models.Sequential()

# Add an LSTM layer with 128 units and input shape (maxlen, len(chars))
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))

# Add a Dense layer with `len(chars)` units and softmax activation
model.add(layers.Dense(len(chars), activation='softmax'))

# Define the optimizer and compile the model
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
# Define a function for sampling the output probabilities
def sample(preds, temperature=1.0):
  # Convert the predictions to float64 data type
  preds = np.asarray(preds).astype('float64')
  # Apply the temperature parameter to the log of the predictions
  preds = np.log(preds) / temperature
  # Compute the exponential of the modified predictions and normalize them
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  # Sample a probability distribution using the multinomial function
  probas = np.random.multinomial(1, preds, 1)
  # Return the index of the highest probability value
  return np.argmax(probas)

# Iterate over the epochs
for epoch in range(1, 60):
  print(f'\n====== EPOCH: {epoch} ======')
  
  # Train the model for one epoch
  model.fit(x, y, batch_size=128, epochs=1)


====== EPOCH: 1 ======
872/872 [==============================] - 12s 5ms/step - loss: 2.1180

====== EPOCH: 2 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.6591

====== EPOCH: 3 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.5058

====== EPOCH: 4 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.4200

====== EPOCH: 5 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.3625

====== EPOCH: 6 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.3210

====== EPOCH: 7 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.2879

====== EPOCH: 8 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.2630

====== EPOCH: 9 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.2413

====== EPOCH: 10 ======
872/872 [==============================] - 4s 5ms/step - loss: 1.2217

====== EPOCH: 11 ======
872/872 [========================

In [9]:
# Iterate over 20 sets of generated text
for i in range(0, 20):
  print(f'\n====== GENERATED TEXT #{i+1}:\n')
 
  # Select a random text seed
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  
  # Print the seed
  print('--- Generating with seed: "' + generated_text + '"')
  
  # Iterate over different temperature values
  for temperature in [0.2, 0.5, 1.0, 1.2]: # Tries a range of different sampling temperatures
    print('\n------ temperature:', temperature, '\n')

    # Print the seed
    sys.stdout.write(generated_text)
    
    # Generate 400 characters
    for i in range(400): # Generates 400 characters, starting from seed text
      # One-hot encode the generated characters so far
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
      
      # Predict the next character using the model and the temperature parameter
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature)
      next_char = chars[next_index]
      
      # Append the next character to the generated text and remove the first character
      generated_text += next_char
      generated_text = generated_text[1:]
      sys.stdout.write(next_char)
    
    print()


====== GENERATED TEXT #1:

--- Generating with seed: " but they should have given him self-defence. it wasn’t a sq"

------ temperature: 0.2 

 but they should have given him self-defence. it wasn’t a square a coment, and then he was a chance to their terms of the street. and they walked around their horses.

“that’s street, but he did not seemed to their terred at the war dance sall and stopped toward the war dance saloon. the wall of the bank had see in the stable, and they walked and stopped at the war dance salters and was a chance.

“than was the store and then their horses in their horses i

------ temperature: 0.5 


“than was the store and then their horses in their horses in the back liffes of their search and still lifted her paral low. “i didn’t see that men out at the high to come ben kelton tony high a corral lips and cultus softly. it was their horses mass went to cause and the store, and there was a still fired his father. “i don’t see that hell’ was a good mistake on t